<a href="https://colab.research.google.com/github/Steve-YJ/Extracting-Important-Feature-of-Malimg-using-VAE/blob/master/01_MNIST_AAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reference
* god....!: https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/aae/aae.py

### What I want to do
* run AAE 
* practicing train

* from 20.07.15.wed<br>
* prace & make my own work flow<br>
    * continue: https://github.com/Steve-YJ/Colab_Exercise/blob/master/Again_Training_Exp05_Just_20Epochs.ipynb

## 00. Mount Drive

* If you use colab, first mount driver

First, always check if your GPU is possible or not


In [2]:
! nvidia-smi

Wed Jul 15 11:48:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

set 'autoreload'

In [3]:
%load_ext autoreload
%autoreload 2

Mount Drive

In [4]:
# drive mount

from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
! pwd

/content


In [6]:
%cd drive/My\ Drive/Post_InfoSec_Exp
! pwd

/content/drive/My Drive/Post_InfoSec_Exp
/content/drive/My Drive/Post_InfoSec_Exp


## 01. Library Import

In [7]:
import argparse
import os
import numpy as np
import math
import itertools

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch


In [8]:
os.makedirs("images", exist_ok=True)

In [9]:
img_shape = (1, 32, 32)

cuda = True if torch.cuda.is_available() else False

## 02. Data Preprocessing
* Load dataset
* preprocess it
    * transforms
    * make custom dataset
    * train_test split
    * make train loader, test loader

* work Flow
    * transforms module 사용해서 image data compose
        * size 조정 및 normalize, tensor 변환
        * Image Folder를 이용해 dataload
        * dataset split: train dataset, test dataset
        * DataLoader로 batch단위 dataset 불러오기

In [10]:
# configure data loader

os.makedirs("./data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(32), 
             transforms.ToTensor(),
             transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size = 64,
    shuffle=True
)

# 03. Define Adversarial Variational AutoEncoder Class

* Encoder Class
* Decoder Class
* Discriminator Class
* Optimizer

### 3-1. Encoder Class

In [11]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.mu = nn.Linear(512, 10)      # 10: latent dim
        self.logvar = nn.Linear(512, 10)  # 10: latent dim

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        x = self.model(img_flat)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = reparameterization(mu, logvar)
        return z 

### 3-2. Decoder

In [12]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        img_flat = self.model(z)
        img = img_flat.view(img_flat.shape[0], *img_shape)
        return img

### 3-3. Discriminator

In [25]:
'''
class Discriminator(nn.Module):
    def __int__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity
'''
# missing i... ;;

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity

### reparameterization module

In [14]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), 10))))
    z = sampled_z * std + mu
    return z

### Define Loss Function

In [27]:
# Use binary cross-entropy loss
adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()

In [28]:
discriminator.parameters()

<generator object Module.parameters at 0x7fef981a9678>

### Initialize Generator & Discriminator



In [30]:
# Initialize generator and discriminator
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()

if cuda:
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()

### Optimizer

In [29]:
optimizer_G = torch.optim.Adam(
    itertools.chain(encoder.parameters(), decoder.parameters()), lr=0.0002, betas=(0.5, 0.999)
)
# optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [35]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    # Sample noise
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, 10))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)

## 04. Training

In [ ]:
for epoch in range(200):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)

        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * pixelwise_loss(
            decoded_imgs, real_imgs
        )

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as discriminator ground truth
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 10))))

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, 200, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % 400 == 0:
            sample_image(n_row=10, batches_done=batches_done)

[Epoch 0/200] [Batch 0/938] [D loss: 0.697496] [G loss: 0.916604]
[Epoch 0/200] [Batch 1/938] [D loss: 0.697014] [G loss: 0.914032]
[Epoch 0/200] [Batch 2/938] [D loss: 0.697648] [G loss: 0.914530]
[Epoch 0/200] [Batch 3/938] [D loss: 0.685569] [G loss: 0.919160]
[Epoch 0/200] [Batch 4/938] [D loss: 0.691651] [G loss: 0.917538]
[Epoch 0/200] [Batch 5/938] [D loss: 0.698837] [G loss: 0.914731]
[Epoch 0/200] [Batch 6/938] [D loss: 0.693931] [G loss: 0.914616]
[Epoch 0/200] [Batch 7/938] [D loss: 0.696431] [G loss: 0.915203]
[Epoch 0/200] [Batch 8/938] [D loss: 0.692038] [G loss: 0.915675]
[Epoch 0/200] [Batch 9/938] [D loss: 0.694737] [G loss: 0.914583]
[Epoch 0/200] [Batch 10/938] [D loss: 0.700737] [G loss: 0.920999]
[Epoch 0/200] [Batch 11/938] [D loss: 0.701460] [G loss: 0.913850]
[Epoch 0/200] [Batch 12/938] [D loss: 0.690443] [G loss: 0.912294]
[Epoch 0/200] [Batch 13/938] [D loss: 0.696941] [G loss: 0.916601]
[Epoch 0/200] [Batch 14/938] [D loss: 0.696271] [G loss: 0.914377]
[Epoc